In [1]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from ResNet import *
from run_model import *


import time
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models, transforms
from collections import OrderedDict
from PIL import Image
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.4.0a0+649135b
Torchvision Version:  0.5.0a0


/opt/conda/lib/python3.6/site-packages/torchvision/io/_video_opt.py:17: UserWarning: video reader based on ffmpeg c++ ops not available
  warnings.warn("video reader based on ffmpeg c++ ops not available")


In [2]:
from dataloader import RetinopathyLoader

In [3]:
# Batch size for training (change depending on how much memory you have)
batch_size = 1

train_set = RetinopathyLoader(root='./data/',mode='train')
train_loader = data.DataLoader(dataset=train_set,batch_size=batch_size)
test_set = RetinopathyLoader(root='./data/',mode='test')
test_loader = data.DataLoader(dataset=test_set,batch_size=batch_size)

dataloaders_dict = {'train':train_loader,
                    'val' : test_loader}

> Found 28099 images...
> Found 7025 images...


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet18"

# Number of classes in the dataset
num_classes = 5



# Number of epochs to train for 
num_epochs = 10

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [6]:
def get_pretrained_model(model_name, num_classes,use_pretrained=True):
    model_dict = {
        'resnet18':  models.resnet18,
        'resnet34':  models.resnet34,
        'resnet50':  models.resnet50,
        'resnet101': models.resnet101,
        'resnet152': models.resnet152
    }
    
    model_ft = model_dict[model_name](pretrained=use_pretrained)          
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224
    
    return model_ft, input_size

## Run ResNet pretrained model which is from `torchvision`

In [ ]:
# Initialize the pretrained model for this run
model_ft, input_size = get_pretrained_model(model_name, num_classes=5)


# Setup optimizer
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9)

# Send the model to GPU
model_ft = model_ft.to(device)

# Train and evaluate pretrained model
pretrained_loss_list, pretrained_acc_train_list, pretrained_acc_test_list = \
    run(model_ft, dataloaders_dict, criterion, num_epochs=num_epochs)

In [ ]:
torch.save(model_ft,'./models/vision_pretrained_resnet18')
torch.save(pretrained_loss_list,'./result_list/pretrained_loss_list')
torch.save(pretrained_acc_train_list, './result_list/pretrained_acc_train_list')
torch.save(pretrained_acc_test_list, './result_list/pretrained_acc_test_list')

## Run ResNet model (no pretrained) which is from `torchvision`

In [ ]:
# Initialize the non-pretrained version of the model used for this run
#scratch_model,_ = get_pretrained_model(model_name, num_classes=5, use_pretrained=False)
#scratch_model = scratch_model.to(device)
#scratch_loss_list, scratch_acc_train_list, scratch_acc_test_list = \
#    run(scratch_model, dataloaders_dict, criterion, num_epochs=num_epochs)

In [ ]:
#torch.save(scratch_model,'./models/vision_not_pretrained_resnet18')

## Run ResNet implemented by myself

In [ ]:
MyResNet = ResNet18(num_classes=5)
MyResNet = MyResNet.to(device)

In [ ]:
my_loss_list, my_acc_train_list, my_acc_test_list = \
    run(MyResNet, dataloaders_dict, criterion, num_epochs=num_epochs)


In [ ]:

torch.save(MyResNet,'./models/my_resnet18')
torch.save(my_loss_list,'./result_list/my_loss_list_18')
torch.save(my_acc_train_list, './result_list/my_acc_train_list_18')
torch.save(my_acc_test_list, './result_list/my_acc_test_list_18')

## Comparison

In [ ]:
model_ft = torch.load('./models/vision_pretrained_resnet18')
pretrained_acc_test_list = torch.load('./result_list/pretrained_acc_test_list')

In [ ]:
ohist  = [h.cpu().numpy() for h in pretrained_acc_test_list]
#shist  = [h.cpu().numpy() for h in scratch_acc_test_list]
myhist = [h.cpu().numpy() for h in my_acc_test_list]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
#plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.plot(range(1,num_epochs+1),myhist,label="My model")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

## ResNet 50

In [ ]:
# Initialize the pretrained model for this run
model_ft_50, input_size = get_pretrained_model('resnet50', num_classes=5)
print(model_ft_50)

In [ ]:
# Setup optimizer
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9)

# Send the model to GPU
model_ft_50 = model_ft_50.to(device)

# Train and evaluate pretrained model
pretrained_loss_list_50, pretrained_acc_train_list_50, pretrained_acc_test_list_50 = \
    run(model_ft_50, dataloaders_dict, criterion, num_epochs=5)

In [ ]:
torch.save(model_ft_50,'./models/vision_pretrained_resnet50')
torch.save(pretrained_loss_list_50,'./result_list/pretrained_loss_list_50')
torch.save(pretrained_acc_train_list_50, './result_list/pretrained_acc_train_list_50')
torch.save(pretrained_acc_test_list_50, './result_list/pretrained_acc_test_list_50')

In [ ]:
MyResNet50 = ResNet50(num_classes=5)
MyResNet50 = MyResNet50.to(device)
my_loss_list_50, my_acc_train_list_50, my_acc_test_list_50 = \
    run(MyResNet50, dataloaders_dict, criterion, num_epochs=5)

In [ ]:
torch.save(MyResNet50,'./models/my_resnet50')
torch.save(my_loss_list_50,'./result_list/my_loss_list_50')
torch.save(my_acc_train_list_50, './result_list/my_acc_train_list_50')
torch.save(my_acc_test_list_50, './result_list/my_acc_test_list_50')

In [ ]:
MyResNet101 = ResNet101(num_classes=5)
MyResNet101 = MyResNet101.to(device)
my_loss_list_101, my_acc_train_list_101, my_acc_test_list_101 = \
    run(MyResNet101, dataloaders_dict, criterion, num_epochs=5)

In [ ]:
torch.save(MyResNet101,'./models/my_resnet101')
torch.save(my_loss_list_101,'./result_list/my_loss_list_101')
torch.save(my_acc_train_list_101, './result_list/my_acc_train_list_101')
torch.save(my_acc_test_list_101, './result_list/my_acc_test_list_101')

# Run ResNet152

In [7]:
MyResNet152 = ResNet152(num_classes=5)
MyResNet152 = MyResNet152.to(device)
print(MyResNet152)

ResNet152(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BottleneckBlock(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu3

In [ ]:

my_loss_list_152, my_acc_train_list_152, my_acc_test_list_152 = \
    run(MyResNet152, dataloaders_dict, criterion, num_epochs=5)

Epoch 1/5
----------


In [ ]:
torch.save(MyResNet152,'./models/my_resnet152')
torch.save(my_loss_list_152,'./result_list/my_loss_list_152')
torch.save(my_acc_train_list_152, './result_list/my_acc_train_list_152')
torch.save(my_acc_test_list_152, './result_list/my_acc_test_list_152')

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)